<a href="https://colab.research.google.com/github/gisalgs/notebooks/blob/main/POI-point-of-interest-case-study-colab.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Case Study: Accessibility 


The City of Columbus has put together a collection of points of interest for different functional categories. This data can be viewed at their interactive website https://opendata.columbus.gov/datasets/columbus::points-of-interest/explore. The description of the data can be found at https://maps2.columbus.gov/arcgis/rest/services/Schemas/PointsOfInterest/MapServer/10. The raw geojson file has been posted at https://raw.githubusercontent.com/gisalgs/data/refs/heads/master/columbus_points_of_interest.geojson. We will use this geojson file in this tutorial.

The goal of this tutorial is to demonstrate the use of Python for spatial data analysis. More specifically, we will examine how residents in Franklin county can access some of the services represented in this data set. Here, we must realize that the data is incomplete because it may not fully cover areas outside the city of Columbus and some points of interest may not be included in the data either. The main purpose is the demonstrate the use of data and Python coding to analyze accessibility. 

Most of the code is not directly shown in the original tutorial. The associated video (from the in-class demo) will have a walk-through of most of the processes. After finishing this tutorial, students should answer the following questions (with code):

1. How many types of points of interest are represented in the data and how many instances are there in each type?
2. Where are the POIs distributed on a map? 
3. If we redistribute the population of Franklin County evenly into cells in each census block group, who does the population distribution look like on the map?
4. How accessibility to a certain service (medical) looks like on the map?
5. What are the shortcomings of the analysis presented in this tutorial?


## Accessibility: a very short introduction

We will use this data to compute the accessibility to various services and we will use health as an example. Accessibility is a big topic and we will focus on a particular approach called the two-step floating catchment area (2SFCA) method (Luo and Wang 2003). Let's assume we want to compute the accessibility at location $i$ to various services provided at locations denoted as $j$. We call $i$ the demand location and $j$ the supply location. The first step is to compute the supply to demand ratio at each supply location $j$ as follows:

$R_j = \Large\frac{S_j}{\sum_{k \in (d_{jk} < d_0)} D_k}$

where $R_j$ is the ratio, $S_j$ is the capacity of the supply at location $j$, $d_{kj}$ is the demand from location $k$ that require supply from $j$, and $d_0$ is a distance buffer that defines the catchment of supply $j$. We use $(d_{jk} < d_0)$ to a set of locations ($k$) whose distance to $j$ is smaller than $d_0$. We can replace this set with any set that can be used to define the catchment and therefore the catchment doesn't have to be a circular buffer but can be of any shape. The above equation basically counts all the demand within the catchment (defined by a radius of $d_0$) and split the supply among them. Here, the demand is weighted equally regardless of how far it is from the supply. It is reasonable, as many researchers have done so, to further weight the demand by some kind of  impedance function (e.g., distance decay function). This, however, is dependent on the actual demand and supply. 

The second step, now that we have worked out the supply side, is to actually compute the accessibility at the demand side by simply add up all the supply to demand ratios that are within the catchment radius of the demand location:

$\Large A_i = \sum_{j\in (d_{ij}<d_0)}{R_j}$

where $A_i$ is the accessibility at demand $i$ and we use $(d_{ij} < d_0)$ to denote the supply locations ($j$) that are within the distance of $d_0$. 

In an application, we have many demand locations to calculate and we will "float" this calculation over the space so that we get the accessibility measure of all the locations of our interest. 

**Work cited**

Luo, W., & Wang, F. (2003). Measures of spatial accessibility to health care in a GIS environment: synthesis and a case study in the Chicago region. Environment and planning B: planning and design, 30(6), 865-884.

In [ ]:
!rm -rf geom
!git clone https://github.com/gisalgs/geom.git

In [ ]:
import urllib.request as request
import json 

In [ ]:
url = 'https://raw.githubusercontent.com/gisalgs/data/refs/heads/master/columbus_points_of_interest.geojson'
with request.urlopen(url) as response:
    poi = json.loads(response.read())

len(poi['features'])

In [ ]:
url = 'https://raw.githubusercontent.com/gisalgs/data/refs/heads/master/blockgrps_pop_franklin_2.geojson'
with request.urlopen(url) as response:
    blkgrps = json.loads(response.read())

len(blkgrps['features'])

In [ ]:
blkgrps['features'][0]['properties'].keys()

In [ ]:
# check how many are multiplygons -- all of them!

sum([f['geometry']['type'] == 'MultiPolygon' for f in blkgrps['features']])

In [ ]:
# How many have multiple parts

# there are two 

sum([len(f['geometry']['coordinates'])>1 for f in blkgrps['features']])

In [ ]:
# holes?

# How many have holes -- forget about the two with multiple parts

# there is one

sum([len(f['geometry']['coordinates'][0])>1 for f in blkgrps['features']])

In [ ]:
for f in blkgrps['features']:
    if len(f['geometry']['coordinates'][0])>1:
        print(f['properties']['TRACT'], f['properties']['BLKGRP'])

Explore the types of geometry in the POI data.

In [ ]:
geom_types = []
for p in poi['features']:
    t = p['geometry']['type']
    if t not in geom_types:
        geom_types.append(t)

geom_types

Now we know all the features are of the same type: Point. Let's know explore the attributes a little bit before we draw the map.

In [ ]:
p['properties']

In [ ]:
poi_types = {}
for p in poi['features']:
    t = p['properties']['POI_TYPE']
    if t not in poi_types:
        poi_types[t] = 1
    else:
        poi_types[t] += 1

poi_types